# Lab 3: Spark MLlib - Machine Learning

## 🎯 **Learning Objectives:**
- Master Spark MLlib machine learning library
- Learn ML pipeline concepts and workflows
- Practice feature engineering techniques
- Understand model training and evaluation
- Implement real-world ML use cases

## 📚 **Key Concepts:**
1. **MLlib**: Spark's machine learning library
2. **ML Pipelines**: End-to-end ML workflows
3. **Feature Engineering**: Data transformation for ML
4. **Model Training**: Supervised and unsupervised learning
5. **Model Evaluation**: Performance metrics and validation

## 🏗️ **Architecture Overview:**
```
┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│   Raw Data      │───▶│ Feature Pipeline │───▶│   ML Models     │
│   (CSV/JSON)    │    │ • Transformers   │    │ • Classification│
│                 │    │ • Estimators    │    │ • Regression    │
└─────────────────┘    └──────────────────┘    │ • Clustering   │
         │                        │               └─────────────────┘
         ▼                        ▼                        │
┌─────────────────┐    ┌──────────────────┐               ▼
│ Data Preprocessing│    │ Feature Selection│    ┌─────────────────┐
│ • Cleaning       │    │ • Scaling        │    │ Model Evaluation│
│ • Validation    │    │ • Encoding       │    │ • Metrics       │
└─────────────────┘    └──────────────────┘    │ • Validation   │
                                                └─────────────────┘
```

## 🤖 **ML Use Cases:**
- **Classification**: Customer segmentation, fraud detection
- **Regression**: Price prediction, demand forecasting
- **Clustering**: Market segmentation, anomaly detection
- **Recommendation**: Product recommendations, content filtering


In [ ]:
# Install and Import Dependencies
%pip install pyspark findspark pandas numpy pyarrow scikit-learn matplotlib seaborn

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import *
from pyspark.ml.classification import *
from pyspark.ml.regression import *
from pyspark.ml.clustering import *
from pyspark.ml.evaluation import *
from pyspark.ml.tuning import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import random

print("✅ Dependencies installed and imported successfully!")


In [ ]:
# Initialize Spark Session for MLlib
spark = SparkSession.builder \
    .appName("SparkMLlibLab") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .getOrCreate()

# Set log level to reduce verbosity
spark.sparkContext.setLogLevel("WARN")

print("🚀 Spark MLlib Session initialized successfully!")
print(f"📊 Spark Version: {spark.version}")
print(f"🔗 Master URL: {spark.sparkContext.master}")
print(f"🤖 MLlib Version: {spark.version}")


In [ ]:
# Create Sample ML Datasets
print("📊 Creating sample datasets for ML experiments...")

# 1. Customer Classification Dataset
print("\n👥 Creating customer classification dataset...")

customer_data = []
customer_types = ['Premium', 'Standard', 'Basic']
regions = ['North', 'South', 'East', 'West']
products = ['Electronics', 'Clothing', 'Books', 'Home', 'Sports']

for i in range(1000):
    age = random.randint(18, 80)
    income = random.randint(20000, 150000)
    spending = random.randint(100, 5000)
    
    # Create features that influence customer type
    if income > 80000 and spending > 2000:
        customer_type = 'Premium'
    elif income > 40000 and spending > 800:
        customer_type = 'Standard'
    else:
        customer_type = 'Basic'
    
    customer_data.append({
        'customer_id': f'CUST_{i+1:04d}',
        'age': age,
        'income': income,
        'spending': spending,
        'region': random.choice(regions),
        'product_category': random.choice(products),
        'customer_type': customer_type,
        'satisfaction_score': random.randint(1, 10),
        'years_customer': random.randint(1, 20)
    })

# 2. Sales Regression Dataset
print("\n💰 Creating sales regression dataset...")

sales_data = []
for i in range(800):
    # Features that influence sales
    marketing_budget = random.randint(1000, 50000)
    price = random.uniform(10, 500)
    season = random.choice(['Spring', 'Summer', 'Fall', 'Winter'])
    promotion = random.choice(['None', 'Discount', 'BOGO', 'Free_Shipping'])
    
    # Calculate sales based on features
    base_sales = marketing_budget * 0.1 + random.randint(100, 1000)
    
    if season == 'Summer':
        base_sales *= 1.2
    elif season == 'Winter':
        base_sales *= 1.1
    
    if promotion == 'Discount':
        base_sales *= 1.3
    elif promotion == 'BOGO':
        base_sales *= 1.5
    elif promotion == 'Free_Shipping':
        base_sales *= 1.1
    
    # Add some noise
    sales = base_sales + random.randint(-200, 200)
    sales = max(0, sales)  # Ensure non-negative
    
    sales_data.append({
        'product_id': f'PROD_{i+1:04d}',
        'marketing_budget': marketing_budget,
        'price': round(price, 2),
        'season': season,
        'promotion': promotion,
        'sales': round(sales, 2),
        'competitor_price': round(price * random.uniform(0.8, 1.2), 2),
        'inventory_level': random.randint(10, 1000)
    })

# 3. Customer Clustering Dataset
print("\n🔍 Creating customer clustering dataset...")

clustering_data = []
for i in range(600):
    # Generate features for clustering
    age = random.randint(18, 70)
    income = random.randint(20000, 120000)
    spending_frequency = random.randint(1, 30)  # times per month
    avg_order_value = random.randint(20, 500)
    
    clustering_data.append({
        'customer_id': f'CLUST_{i+1:04d}',
        'age': age,
        'income': income,
        'spending_frequency': spending_frequency,
        'avg_order_value': avg_order_value,
        'online_visits': random.randint(1, 100),
        'mobile_app_usage': random.randint(0, 50),
        'customer_service_calls': random.randint(0, 10)
    })

print(f"\n✅ Sample datasets created:")
print(f"   👥 Customer classification: {len(customer_data)} records")
print(f"   💰 Sales regression: {len(sales_data)} records")
print(f"   🔍 Customer clustering: {len(clustering_data)} records")


In [ ]:
# Create Spark DataFrames for ML
print("🔄 Creating Spark DataFrames for ML experiments...")

# Convert to Spark DataFrames
customers_df = spark.createDataFrame(customer_data)
sales_df = spark.createDataFrame(sales_data)
clustering_df = spark.createDataFrame(clustering_data)

print("\n📊 Customer Classification DataFrame:")
customers_df.printSchema()
customers_df.show(5)

print("\n💰 Sales Regression DataFrame:")
sales_df.printSchema()
sales_df.show(5)

print("\n🔍 Customer Clustering DataFrame:")
clustering_df.printSchema()
clustering_df.show(5)

print(f"\n✅ ML DataFrames created successfully!")
print(f"   👥 Customers: {customers_df.count()} records")
print(f"   💰 Sales: {sales_df.count()} records")
print(f"   🔍 Clustering: {clustering_df.count()} records")


## Exercise 1: Customer Classification

### 🎯 **Learning Objectives:**
- Master classification algorithms in MLlib
- Learn feature engineering for ML
- Practice ML pipeline creation
- Understand model evaluation metrics

### 📚 **Key Concepts:**
1. **Classification**: Predicting categorical outcomes
2. **Feature Engineering**: Preparing data for ML
3. **ML Pipelines**: End-to-end ML workflows
4. **Model Evaluation**: Performance metrics and validation


In [ ]:
# Exercise 1: Customer Classification
print("🎯 Exercise 1: Customer Classification")

print("\n🔧 Setting up feature engineering pipeline...")

# Define feature columns
feature_columns = ['age', 'income', 'spending', 'satisfaction_score', 'years_customer']
categorical_columns = ['region', 'product_category']

# Create feature vector assembler
assembler = VectorAssembler(
    inputCols=feature_columns,
    outputCol="features"
)

# Create string indexers for categorical variables
region_indexer = StringIndexer(
    inputCol="region",
    outputCol="region_index"
)

product_indexer = StringIndexer(
    inputCol="product_category", 
    outputCol="product_index"
)

# Create label indexer for target variable
label_indexer = StringIndexer(
    inputCol="customer_type",
    outputCol="label"
)

# Create Random Forest Classifier
rf_classifier = RandomForestClassifier(
    featuresCol="features",
    labelCol="label",
    numTrees=100,
    maxDepth=10,
    seed=42
)

print("\n📊 Building ML pipeline...")

# Create ML pipeline
pipeline = Pipeline(stages=[
    region_indexer,
    product_indexer,
    label_indexer,
    assembler,
    rf_classifier
])

print("\n🔄 Splitting data into train/test sets...")

# Split data into training and test sets
train_data, test_data = customers_df.randomSplit([0.8, 0.2], seed=42)

print(f"   📊 Training set: {train_data.count()} records")
print(f"   📊 Test set: {test_data.count()} records")

print("\n🚀 Training the model...")

# Train the model
model = pipeline.fit(train_data)

print("✅ Model training completed!")

print("\n📊 Making predictions on test set...")

# Make predictions
predictions = model.transform(test_data)

print("\n📈 Model evaluation results:")

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction",
    metricName="accuracy"
)

accuracy = evaluator.evaluate(predictions)
print(f"   🎯 Accuracy: {accuracy:.4f}")

# Additional metrics
evaluator_f1 = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction", 
    metricName="f1"
)

f1_score = evaluator_f1.evaluate(predictions)
print(f"   📊 F1 Score: {f1_score:.4f}")

print("\n📋 Sample predictions:")
predictions.select("customer_id", "customer_type", "prediction", "probability").show(10)

print("\n✅ Customer classification completed!")
